# Business Metadata Management - Complete Guide

This notebook demonstrates how to create, manage, and apply **Business Metadata** in Microsoft Purview using the CLI.

## What is Business Metadata?

Business Metadata allows you to add **custom attributes** to your Purview entities. Unlike UC Custom Metadata (which is for governance policies), Business Metadata is for enriching your data catalog with custom fields.

## Two Types of Scopes

1. **Business Concept Scope**: Applies to Terms, Domains, Business Rules
2. **Data Asset Scope**: Applies to Tables, Files, Databases, etc.

## What You'll Learn

- ✅ Create Business Metadata groups with attributes
- ✅ List and view existing metadata
- ✅ Understand scopes (Business Concept vs Data Asset)
- ✅ Apply metadata to entities
- ✅ Update and delete metadata groups
- ✅ Advanced patterns (enums, validation)

## Prerequisites

- PVW CLI installed: `pip install pvw-cli`
- Azure authentication: `az login`
- Environment variables: `PURVIEW_ACCOUNT_NAME`, `PURVIEW_ACCOUNT_ID`

## ⚠️ Important: Command Structure

All Business Metadata commands are under the `types` group:

```bash
# ✅ Correct format
py -m purviewcli types <command> [options]

# ❌ Incorrect (missing 'types')
py -m purviewcli <command> [options]
```

**Examples:**
```bash
# List groups
py -m purviewcli types list-business-metadata-groups

# Create metadata
py -m purviewcli types create-business-metadata-def --payload-file file.json

# Read definition
py -m purviewcli types read-business-metadata-def --name GroupName
```

**In this notebook:**
- Commands use `!pvw` or `!py -m purviewcli` with full command structure
- All commands are tested and verified working
- Make sure to include `types` before the command name

---
## Part 1: Viewing Existing Business Metadata
---

### List All Business Metadata Groups (with Scopes)

This command shows all metadata groups with their scope and attribute count.

In [ ]:
# List all Business Metadata groups with scopes
!pvw types list-business-metadata-groups

**Output Explanation:**
- **Group Name**: The metadata group name
- **Scope**: Where it can be applied
  - `Business Concept` = Terms, Domains, Business Rules
  - `Data Asset` = Tables, Files, Databases
  - `Universal` = Everything
- **Attributes**: Number of custom fields in the group

### List All Business Attributes (Detailed View)

See individual attributes with their groups and scopes.

In [ ]:
# List all attributes with detailed information
!pvw types list-business-attributes

### Read Specific Group Definition

Get the complete JSON definition of a specific group.

In [ ]:
# Read the "Global" group definition
!pvw types read-business-metadata-def --name Global

### View with UC Metadata List (Automatic Fallback)

The `uc metadata list` command automatically shows Business Metadata when UC Custom Metadata is empty.

In [ ]:
# View metadata with automatic fallback
!pvw uc metadata list

**Tip:** Use `--no-fallback` to see only UC metadata:
```bash
pvw uc metadata list --no-fallback
```

---
## Part 2: Creating Business Metadata Groups
---

### Example 1: Governance Metadata (Business Concepts)

Create a metadata group for tracking governance information on **Terms and Domains**.

**Attributes:**
- DataOwner (string)
- ComplianceStatus (string)
- ReviewDate (date)

In [ ]:
# First, let's create the JSON template
import json

governance_template = {
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "Governance",
            "description": "Governance and compliance metadata for business concepts",
            "attributeDefs": [
                {
                    "name": "DataOwner",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {
                        "maxStrLength": "200"
                    }
                },
                {
                    "name": "ComplianceStatus",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {
                        "maxStrLength": "50"
                    }
                },
                {
                    "name": "ReviewDate",
                    "typeName": "date",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"domain:*\",\"businessConcept:*\"]}"
            }
        }
    ]
}

# Save to file
with open('temp_governance.json', 'w') as f:
    json.dump(governance_template, f, indent=2)

print("✅ Governance template created: temp_governance.json")
print("\n📋 Template structure:")
print(json.dumps(governance_template, indent=2))

**Key Points:**
- `"category": "BUSINESS_METADATA"` - Required for all business metadata
- `"name": "Governance"` - The group name (must be unique)
- `"attributeDefs"` - Array of custom attributes
- `"dataGovernanceOptions"` - Defines the scope
  - `"domain:*"` - Applies to all domains
  - `"businessConcept:*"` - Applies to all business concepts (terms)

### Validate Before Creating (Dry-Run)

Always validate your JSON before creating metadata.

In [ ]:
# Validate with dry-run
!pvw types create-business-metadata-def --payload-file temp_governance.json --dry-run --validate

### Create the Metadata Group

If validation passes, create the metadata group.

In [ ]:
# Create the Governance metadata group
!pvw types create-business-metadata-def --payload-file temp_governance.json

### Verify Creation

Check that the new group appears in the list.

In [ ]:
# Verify the new group
!pvw types list-business-metadata-groups

---
### Example 2: Data Quality Metadata (Data Assets)

Create metadata for tracking quality metrics on **Tables and Files**.

**Attributes:**
- QualityScore (integer 0-100)
- LastValidated (date)
- ValidationNotes (string)

In [ ]:
# Create DataQuality template
quality_template = {
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "DataQuality",
            "description": "Data quality metrics for data assets",
            "attributeDefs": [
                {
                    "name": "QualityScore",
                    "typeName": "int",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {
                        "minValue": "0",
                        "maxValue": "100"
                    }
                },
                {
                    "name": "LastValidated",
                    "typeName": "date",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "ValidationNotes",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {
                        "maxStrLength": "1000"
                    }
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"dataset:*\"]}"
            }
        }
    ]
}

# Save to file
with open('temp_quality.json', 'w') as f:
    json.dump(quality_template, f, indent=2)

print("✅ DataQuality template created")
print("\n📋 Key difference: Scope is 'dataset:*' (Data Asset)")

**Note the Scope Difference:**
- Governance: `"domain:*", "businessConcept:*"` → Business Concepts
- DataQuality: `"dataset:*"` → Data Assets (Tables, Files)

In [ ]:
# Create DataQuality metadata group
!pvw types create-business-metadata-def --payload-file temp_quality.json

In [ ]:
# Verify both groups
!pvw types list-business-metadata-groups

---
### Example 3: Privacy Metadata (Business Concepts Only)

Create metadata for privacy classification on **Terms only**.

**Attributes:**
- PrivacyLevel (string)
- PIIContained (boolean)
- DataClassification (string)

In [ ]:
# Create Privacy template
privacy_template = {
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "Privacy",
            "description": "Privacy classification for sensitive terms",
            "attributeDefs": [
                {
                    "name": "PrivacyLevel",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {
                        "maxStrLength": "50"
                    }
                },
                {
                    "name": "PIIContained",
                    "typeName": "boolean",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "DataClassification",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {
                        "maxStrLength": "100"
                    }
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"businessConcept:*\"]}"
            }
        }
    ]
}

# Save and create
with open('temp_privacy.json', 'w') as f:
    json.dump(privacy_template, f, indent=2)

print("✅ Privacy template created")
print("\n📋 Scope: 'businessConcept:*' only (Terms, not Domains)")

In [ ]:
# Create Privacy metadata group
!pvw types create-business-metadata-def --payload-file temp_privacy.json

---
### Example 4: Universal Metadata (Everything)

Create metadata that applies to **both Business Concepts AND Data Assets**.

**Attributes:**
- DocumentationLink (string)
- LastUpdated (date)
- UpdatedBy (string)

In [ ]:
# Create Universal/Documentation template
universal_template = {
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "Documentation",
            "description": "Documentation metadata applicable to all assets and concepts",
            "attributeDefs": [
                {
                    "name": "DocumentationLink",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {
                        "maxStrLength": "500"
                    }
                },
                {
                    "name": "LastUpdated",
                    "typeName": "date",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "UpdatedBy",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {
                        "maxStrLength": "200"
                    }
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"domain:*\",\"businessConcept:*\",\"dataset:*\"]}"
            }
        }
    ]
}

# Save and create
with open('temp_documentation.json', 'w') as f:
    json.dump(universal_template, f, indent=2)

print("✅ Documentation template created")
print("\n📋 Scope: ALL constructs (Universal)")

In [ ]:
# Create Documentation metadata group
!pvw types create-business-metadata-def --payload-file temp_documentation.json

In [ ]:
# View all created groups
!pvw types list-business-metadata-groups

---
## Part 3: Advanced Patterns
---

### Advanced: Metadata with Enums (Dropdowns)

Create metadata with **enum types** for dropdown selections in the Purview UI.

**Use Case:** Status tracking with predefined values.

In [ ]:
# Create metadata with enum types
advanced_template = {
    "enumDefs": [
        {
            "category": "ENUM",
            "name": "StatusEnum",
            "description": "Status values for governance workflows",
            "elementDefs": [
                {"ordinal": 0, "value": "Draft"},
                {"ordinal": 1, "value": "InReview"},
                {"ordinal": 2, "value": "Approved"},
                {"ordinal": 3, "value": "Rejected"},
                {"ordinal": 4, "value": "Archived"}
            ]
        },
        {
            "category": "ENUM",
            "name": "ConfidentialityEnum",
            "description": "Confidentiality levels",
            "elementDefs": [
                {"ordinal": 0, "value": "Public"},
                {"ordinal": 1, "value": "Internal"},
                {"ordinal": 2, "value": "Confidential"},
                {"ordinal": 3, "value": "HighlyConfidential"}
            ]
        }
    ],
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "AdvancedGovernance",
            "description": "Advanced governance with enum dropdowns",
            "attributeDefs": [
                {
                    "name": "Status",
                    "typeName": "StatusEnum",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "Confidentiality",
                    "typeName": "ConfidentialityEnum",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "ApprovedBy",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "200"}
                },
                {
                    "name": "ApprovalDate",
                    "typeName": "date",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"domain:*\",\"businessConcept:*\"]}"
            }
        }
    ]
}

# Save template
with open('temp_advanced.json', 'w') as f:
    json.dump(advanced_template, f, indent=2)

print("✅ Advanced template created with enums")
print("\n📋 Enums defined:")
print("  - StatusEnum: Draft, InReview, Approved, Rejected, Archived")
print("  - ConfidentialityEnum: Public, Internal, Confidential, HighlyConfidential")

In [ ]:
# Create advanced metadata with enums
!pvw types create-business-metadata-def --payload-file temp_advanced.json

**Benefit of Enums:**
- Creates **dropdowns** in Purview UI
- Ensures **data consistency** (only allowed values)
- Better for **reporting and filtering**

---
## Part 4: Data Type Reference
---

### Available Data Types

| Type Name | Description | Example Options |
|-----------|-------------|-----------------|
| `string` | Text field | `"maxStrLength": "500"` |
| `int` | Integer number | `"minValue": "0", "maxValue": "100"` |
| `long` | Long integer | - |
| `float` | Decimal number | - |
| `double` | Double precision decimal | - |
| `date` | Date value | - |
| `boolean` | True/False | - |
| Custom Enum | Dropdown list | Define in `enumDefs` first |

### Data Type Examples

In [ ]:
# Example of each data type
data_types_example = {
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "DataTypeExamples",
            "description": "Examples of all data types",
            "attributeDefs": [
                {
                    "name": "TextField",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "500"}
                },
                {
                    "name": "NumberField",
                    "typeName": "int",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"minValue": "0", "maxValue": "100"}
                },
                {
                    "name": "DecimalField",
                    "typeName": "double",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "DateField",
                    "typeName": "date",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "BooleanField",
                    "typeName": "boolean",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"dataset:*\"]}"
            }
        }
    ]
}

print("📋 Data Type Examples:")
print(json.dumps(data_types_example, indent=2))

---
## Part 5: Scope Reference
---

### Scope Options Explained

**1. Business Concept Scope** (Terms, Domains, Business Rules)
```json
"options": {
  "dataGovernanceOptions": "{\"applicableConstructs\":[\"domain:*\",\"businessConcept:*\"]}"
}
```

**2. Data Asset Scope** (Tables, Files, Databases)
```json
"options": {
  "dataGovernanceOptions": "{\"applicableConstructs\":[\"dataset:*\"]}"
}
```

**3. Universal Scope** (Everything)
```json
"options": {
  "dataGovernanceOptions": "{\"applicableConstructs\":[\"domain:*\",\"businessConcept:*\",\"dataset:*\"]}"
}
```

**4. Legacy Format** (Specific Entity Types)
```json
"options": {
  "applicableEntityTypes": "[\"azure_sql_table\",\"azure_sql_db\",\"hive_table\"]"
}
```

### Scope Decision Guide

| Your Need | Scope | Example Use Case |
|-----------|-------|------------------|
| Enrich business terms | Business Concept | Ownership, definitions, compliance |
| Track data quality | Data Asset | Quality scores, validation dates |
| Add documentation links | Universal | Links applicable to all entities |
| Specific table types only | Legacy (applicableEntityTypes) | Metadata only for SQL tables |

---
## Part 6: Updating and Deleting
---

### Update Existing Metadata Group

To add attributes to an existing group, you must include **all existing attributes** plus new ones.

In [ ]:
# Step 1: Read current definition
!pvw types read-business-metadata-def --name Governance > current_governance.json

print("✅ Current definition saved to: current_governance.json")
print("\n⚠️  IMPORTANT: When updating, include ALL existing attributes plus new ones!")

**Manual Steps to Update:**
1. Read current definition (done above)
2. Edit `current_governance.json` to add new attributes
3. Run update command:
```bash
pvw types update-business-metadata-def --payload-file current_governance.json
```

**Note:** You cannot modify the scope after creation. To change scope, delete and recreate.

### Delete Metadata Group

**⚠️ Warning:** Deleting removes the group and all associated metadata values from entities!

In [ ]:
# Delete a metadata group (commented out for safety)
# !pvw types delete-business-metadata-def --name DataTypeExamples

print("⚠️  To delete a metadata group:")
print("pvw types delete-business-metadata-def --name GroupName")
print("\nThis will remove:")
print("  - The metadata group definition")
print("  - All metadata values applied to entities")
print("  - Cannot be undone!")

---
## Part 7: Applying Metadata to Entities
---

### Method 1: Apply via Purview UI

1. Go to Purview Studio (portal.azure.com)
2. Navigate to **Data Catalog** → Find your entity (table, term, etc.)
3. Click **Edit**
4. Scroll to **Business Metadata** section
5. Select your metadata group
6. Fill in attribute values
7. **Save**

### Method 2: Apply via CLI (Entity Update)

**Note:** Entity update commands require the entity GUID.

In [ ]:
# Example: Find a table entity
# !pvw search query --keywords "your_table_name" --limit 1

# Example: Apply business metadata to entity
# !pvw entity create-business-metadata \
#     --guid "entity-guid-here" \
#     --bm-name "Governance" \
#     --bm-attributes '{"DataOwner":"John Doe","ComplianceStatus":"Approved"}'

print("📋 Apply Metadata Steps:")
print("1. Find entity GUID: pvw search query --keywords 'entity_name'")
print("2. Apply metadata: pvw entity create-business-metadata ...")
print("\nSee entity commands documentation for full details.")

---
## Part 8: Real-World Use Cases
---

### Use Case 1: GDPR Compliance Tracking

Track GDPR compliance for all data assets.

In [ ]:
# GDPR Compliance metadata
gdpr_template = {
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "GDPR",
            "description": "GDPR compliance tracking for data assets",
            "attributeDefs": [
                {
                    "name": "PersonalDataContained",
                    "typeName": "boolean",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "LegalBasis",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "200"}
                },
                {
                    "name": "RetentionPeriod",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "100"}
                },
                {
                    "name": "DataSubjectRights",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "500"}
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"dataset:*\"]}"
            }
        }
    ]
}

print("📋 GDPR Compliance Metadata")
print(json.dumps(gdpr_template, indent=2))

### Use Case 2: Finance Department Metadata

Track cost centers and budget ownership for business terms.

In [ ]:
# Finance metadata
finance_template = {
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "Finance",
            "description": "Finance tracking for business concepts",
            "attributeDefs": [
                {
                    "name": "CostCenter",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "50"}
                },
                {
                    "name": "BudgetOwner",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "200"}
                },
                {
                    "name": "AnnualBudget",
                    "typeName": "double",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                },
                {
                    "name": "FiscalYear",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "10"}
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"domain:*\",\"businessConcept:*\"]}"
            }
        }
    ]
}

print("📋 Finance Department Metadata")
print(json.dumps(finance_template, indent=2))

### Use Case 3: Data Stewardship

Track data stewards and stewardship status across all entities.

In [ ]:
# Data Stewardship metadata
stewardship_template = {
    "businessMetadataDefs": [
        {
            "category": "BUSINESS_METADATA",
            "name": "Stewardship",
            "description": "Data stewardship tracking (universal)",
            "attributeDefs": [
                {
                    "name": "DataSteward",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "200"}
                },
                {
                    "name": "StewardshipStatus",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "50"}
                },
                {
                    "name": "ContactEmail",
                    "typeName": "string",
                    "isOptional": True,
                    "cardinality": "SINGLE",
                    "options": {"maxStrLength": "200"}
                },
                {
                    "name": "LastReviewed",
                    "typeName": "date",
                    "isOptional": True,
                    "cardinality": "SINGLE"
                }
            ],
            "options": {
                "dataGovernanceOptions": "{\"applicableConstructs\":[\"domain:*\",\"businessConcept:*\",\"dataset:*\"]}"
            }
        }
    ]
}

print("📋 Data Stewardship Metadata (Universal Scope)")
print(json.dumps(stewardship_template, indent=2))

---
## Part 9: Best Practices
---

### Best Practices Summary

#### 1. Naming Conventions
- ✅ Use **PascalCase** for group names: `DataQuality`, `Governance`
- ✅ Use **PascalCase** for attributes: `DataOwner`, `QualityScore`
- ✅ Be descriptive and consistent
- ❌ Avoid spaces, special characters, or ambiguous names

#### 2. Scope Selection
- ✅ Use **Business Concept** for term/domain metadata
- ✅ Use **Data Asset** for table/file metadata
- ✅ Use **Universal** only when truly applicable to all
- ❌ Don't use Universal if scope is actually specific

#### 3. Attribute Design
- ✅ Keep attribute names concise but clear
- ✅ Use appropriate data types (string, int, date, boolean)
- ✅ Set realistic `maxStrLength` values
- ✅ Mark fields as `isOptional: True` unless required
- ✅ Use enums for dropdown lists
- ❌ Don't create too many attributes per group (5-10 is ideal)

#### 4. Validation
- ✅ Always use `--dry-run --validate` before creating
- ✅ Validate JSON syntax with online tools
- ✅ Test on non-production environment first
- ❌ Don't create in production without testing

#### 5. Documentation
- ✅ Add meaningful descriptions to groups and attributes
- ✅ Document your metadata schema separately
- ✅ Keep templates in version control (Git)
- ✅ Share templates with team members

#### 6. Maintenance
- ✅ Review metadata usage periodically
- ✅ Clean up unused metadata groups
- ✅ Update descriptions as needs evolve
- ❌ Don't delete groups still in use

---
## Part 10: Troubleshooting
---

### Common Issues and Solutions

#### Issue 1: "Business metadata already exists"
**Cause:** Group name is not unique.

**Solution:**
- Choose a different name, OR
- Delete existing group: `pvw types delete-business-metadata-def --name GroupName`

#### Issue 2: "Invalid JSON"
**Cause:** Syntax error in JSON template.

**Solution:**
- Validate at https://jsonlint.com/
- Check for missing commas, quotes, brackets
- Use `--dry-run --validate` to catch errors

#### Issue 3: Attributes not showing in Purview UI
**Cause:** UI cache delay.

**Solution:**
- Wait 30-60 seconds
- Refresh browser (Ctrl+F5)
- Clear browser cache if persistent

#### Issue 4: Wrong scope applied
**Cause:** Incorrect `applicableConstructs` or `applicableEntityTypes`.

**Solution:**
- Scope cannot be changed after creation
- Delete and recreate with correct scope

#### Issue 5: Cannot update attributes
**Cause:** Update payload missing existing attributes.

**Solution:**
- Read current definition first
- Include ALL existing attributes plus new ones in update payload

#### Issue 6: Enum not showing as dropdown
**Cause:** Enum not defined in same payload, or wrong typeName.

**Solution:**
- Define enum in `enumDefs` section
- Reference exact enum name in `typeName`
- Include both `enumDefs` and `businessMetadataDefs` in payload

#### Issue 7: "No such command" error

**Cause:** Missing the `types` group in command structure.

**Error message:**
```
Error: No such command 'list-business-metadata-groups'.
```

**Solution:**
All Business Metadata commands require the `types` group:

```bash
# ❌ Incorrect
py -m purviewcli list-business-metadata-groups

# ✅ Correct
py -m purviewcli types list-business-metadata-groups
```

**Command Structure:**
```
py -m purviewcli <group> <command> [options]
                  ^^^^^^  ^^^^^^^^
                  types   specific-command
```

**All Business Metadata commands:**
- `py -m purviewcli types list-business-metadata-groups`
- `py -m purviewcli types list-business-attributes`
- `py -m purviewcli types create-business-metadata-def`
- `py -m purviewcli types read-business-metadata-def`
- `py -m purviewcli types update-business-metadata-def`
- `py -m purviewcli types delete-business-metadata-def`

---
## Part 11: Command Reference
---

### Complete Command Reference

#### Create Commands
```bash
# Validate before creating
pvw types create-business-metadata-def --payload-file <file.json> --dry-run --validate

# Create metadata group
pvw types create-business-metadata-def --payload-file <file.json>

# Create with output to file
pvw types create-business-metadata-def --payload-file <file.json> --output-file result.json
```

#### List Commands
```bash
# List all groups with scopes (table format)
pvw types list-business-metadata-groups

# List all groups (JSON format)
pvw types list-business-metadata-groups --output json

# List all attributes with details
pvw types list-business-attributes

# List attributes (JSON format)
pvw types list-business-attributes --output json
```

#### Read Commands
```bash
# Read by name
pvw types read-business-metadata-def --name GroupName

# Read by GUID
pvw types read-business-metadata-def --guid <guid>
```

#### Update Commands
```bash
# Update existing group
pvw types update-business-metadata-def --payload-file <file.json>
```

#### Delete Commands
```bash
# Delete metadata group
pvw types delete-business-metadata-def --name GroupName
```

#### UC Integration (Automatic Fallback)
```bash
# View with automatic fallback
pvw uc metadata list

# Force fallback
pvw uc metadata list --fallback

# Disable fallback (UC only)
pvw uc metadata list --no-fallback

# JSON output
pvw uc metadata list --output json
```

---
## Part 12: Summary and Next Steps
---

### What You've Learned

✅ **Viewing Metadata**
- List groups with scopes
- List attributes with details
- Read specific group definitions
- View with UC fallback

✅ **Creating Metadata**
- Business Concept scope (Terms, Domains)
- Data Asset scope (Tables, Files)
- Universal scope (Everything)
- Advanced patterns with enums

✅ **Data Types**
- String, int, long, float, double, date, boolean
- Custom enums for dropdowns
- Validation options (maxStrLength, minValue, maxValue)

✅ **Scopes**
- Business Concept: `domain:*`, `businessConcept:*`
- Data Asset: `dataset:*`
- Universal: All constructs
- Legacy: Specific entity types

✅ **Best Practices**
- Naming conventions
- Scope selection
- Validation before creation
- Documentation and maintenance

### Next Steps

1. **Create Your First Metadata Group**
   - Start with a simple group (3-5 attributes)
   - Use Business Concept or Data Asset scope
   - Test in non-production first

2. **Apply Metadata to Entities**
   - Use Purview UI to apply metadata values
   - Track ownership, quality, compliance

3. **Explore Advanced Patterns**
   - Create metadata with enums
   - Use validation options
   - Combine multiple metadata groups

4. **Integrate into Workflows**
   - Use PowerShell/Python scripts for bulk operations
   - Track metadata in reports
   - Automate metadata application

### Additional Resources

- **Complete Guide:** `doc/guides/create-business-metadata.md`
- **Templates:** `templates/business_metadata_*.json`
- **Quick Start:** `quick_start_business_metadata.bat`
- **Scope Guide:** `doc/guides/business-metadata-scopes.md`
- **UC vs Business Metadata:** `doc/guides/uc-custom-metadata-role.md`

### Questions?

- Check documentation: `doc/guides/`
- View templates: `templates/`
- Run interactive wizard: `quick_start_business_metadata.bat`

---
## Cleanup (Optional)
---

In [ ]:
# Clean up temporary files
import os

temp_files = [
    'temp_governance.json',
    'temp_quality.json',
    'temp_privacy.json',
    'temp_documentation.json',
    'temp_advanced.json',
    'current_governance.json'
]

for file in temp_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"✅ Removed: {file}")

print("\n🧹 Cleanup complete!")

---
## End of Notebook

**Happy Business Metadata Management! 🎉**

For more examples, see other notebooks in `samples/notebooks (plus)/`